# Import Libraries

In [110]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

ModuleNotFoundError: No module named 'xgboost'

# Read Data

In [74]:
df = pd.read_csv('Tweets.csv')

In [75]:
df.tail()

tweet_id airline_sentiment  airline_sentiment_confidence  \
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativereason_confidence   airline  \
14635                     NaN                     0.0000  American   
14636  Customer Service Issue                     1.0000  American   
14637                     NaN                        NaN  American   
14638  Customer Service Issue                     0.6659  American   
14639                     NaN                     0.0000  American   

      airline_sentiment_gold             name negativereason_gold  \
14635                    NaN  KristenReenders                 NaN   
14636                    NaN         itsropes                 NaN   
14637                    NaN         sanyabun                 NaN   
14638                    NaN       SraJackson                 NaN   
14639                    NaN        daviddtwu                 NaN   

       retweet_count                                               text  \
14635              0  @AmericanAir thank you we got on a different f...   
14636              0  @AmericanAir leaving over 20 minutes Late Flig...   
14637              0  @AmericanAir Please bring American Airlines to...   
14638              0  @AmericanAir you have my money, you change my ...   
14639              0  @AmericanAir we have 8 ppl so we need 2 know h...   

      tweet_coord              tweet_created tweet_location  \
14635         NaN  2015-02-22 12:01:01 -0800            NaN   
14636         NaN  2015-02-22 11:59:46 -0800          Texas   
14637         NaN  2015-02-22 11:59:15 -0800  Nigeria,lagos   
14638         NaN  2015-02-22 11:59:02 -0800     New Jersey   
14639         NaN  2015-02-22 11:58:51 -0800     dallas, TX   

                    user_timezone  
14635                         NaN  
14636                         NaN  
14637                         NaN  
14638  Eastern Time (US & Canada)  
14639                         NaN

# Preprocessing

## Select only text and airline_sentiment

In [76]:
df = df[['text','airline_sentiment']]

In [77]:
df.tail()

text airline_sentiment
14635  @AmericanAir thank you we got on a different f...          positive
14636  @AmericanAir leaving over 20 minutes Late Flig...          negative
14637  @AmericanAir Please bring American Airlines to...           neutral
14638  @AmericanAir you have my money, you change my ...          negative
14639  @AmericanAir we have 8 ppl so we need 2 know h...           neutral

## Map label into integers

In [78]:
df['airline_sentiment'] = df['airline_sentiment'].replace({'positive': 2, 'neutral': 1, 'negative':0})

C:\Users\Asty\AppData\Local\Temp\ipykernel_184468\1813139062.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['airline_sentiment'] = df['airline_sentiment'].replace({'positive': 2, 'neutral': 1, 'negative':0})


## Printing all text to observe data

In [79]:
for text in df['text']:
    print(text)

@VirginAmerica What @dhepburn said.
@VirginAmerica plus you've added commercials to the experience... tacky.
@VirginAmerica I didn't today... Must mean I need to take another trip!
@VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
@VirginAmerica and it's a really big bad thing about it
@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.
it's really the only bad thing about flying VA
@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)
@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP
@virginamerica Well, I didn't…but NOW I DO! :-D
@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.
@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24
@VirginAmerica I &lt;3 pretty graphics. so much better than minimal iconography. :D
@V

@united remember in business, a pleasant experience, means you will tell a friend. A bad experience means you will tell everyone you see
@united let me assure you my travel time is 4h2m not 5h2m #timezones #accuratetraveltimes #3rdtimethishashappened http://t.co/e0C9bI09cf
@united U BUMS HOW DO U LOSE THE BIGGEST BAG ON THE FLIGHT
@united is to airlines as @comcast @XFINITY is to cable/internet
@united customers ✅. Customer service ❌ http://t.co/qP6Aw3nLIP
@united great decision making skills flight ops IAH bag makes a 10 minute connection and I don't, flight pushed early. Disappointed!!!!
Meanwhile, they fucked my flight. @united: The festivities are already in full swing at the United Fairway Club, overlooking the 17th hole..
@united in the process of purchasing ticket, when prices changed. unbelievable. #RipOffs #PriceDiscrimination
@united although, I am stranded in Chicago O'Hare for another 8 hours. Any chance your cracker-jack service team could provide a meal?
@united All fligh

@SouthwestAir I "heart" Southwest but those commercials aimed to satisfy a nano-smattering of travelers to few destinations.
@SouthwestAir I urgently need two receipts sent but CS is closed. Can you help?
@SouthwestAir sent
@SouthwestAir Any ETD for SWA1004 from PHL? Appears delayed but no info available at gate.
@SouthwestAir can you tell me why you Cancelled Flightled the flight vs delay
@SouthwestAir my bday was yesterday and my girl gave birth to my first born and I couldn't get a courtesy flight change from Lynn... THANKS!
^CB how do you know? @SouthwestAir
@SouthwestAir no need, I know the status, you Cancelled Flightled instead of delaying the flight.
@SouthwestAir weather where? And at what time Cancelled Flighted? No I can't because meeting was today.
@SouthwestAir the flight isn't that empty.... Is this rude or not? http://t.co/4RL0p5JChB
@SouthwestAir - We left iPad in a seat pocket.  Filed lost item report. Received it exactly 1 week Late Flightr.  Is that a record?  #unbel

@SouthwestAir have you considered adding the "we'll call you back when we have someone free" feature to your support line?
@SouthwestAir I've been on hold with customer service for over an hour. Can you help?!
@SouthwestAir I've been on hold for an hour now. 59:57 as I type this. Ridiculous! All I need is the link to the chart that has routes, time
@SouthwestAir sent
@SouthwestAir Flight Cancelled Flightled, reFlight Booking Problems online has not worked, on my second round of hold (3 hrs now). Any other options?
@SouthwestAir kudos to the #RSW CS crew for re-routing PAX to alleviate Over sale due to down graded EQP 800 now 500
@SouthwestAir trying to fly out of Nashville tomorrow. How is it looking?
@SouthwestAir do you think the flights out of Nashville will be Cancelled Flighted tomorrow?
@SouthwestAir do you not try to get customers to their destinations? No help to get on another airline if you had no flights for 3 days?
@SouthwestAir I'm an A-list Preferred customer and very loy

@USAirways are people going to answer our calls?
@USAirways pilot forgets to show up to work and Ricky J (gate agent) gives customers attitude. #usairwaysfail
@USAirways : You Make the Reservation; We'll Make the Excuses! #usairwaysfail
@USAirways Thank You for your empathy there is more to this story than just the luggage but thank you for reaching out
@USAirways is utterly unreliable! They didn't schedule a pilot for their US624 flight out of Seattle - flights are delayed #usairwaysfail
@USAirways yes but again valuable time has been lost
@USAirways I don't like being lied to. No bad weather in dc md va. I've been stuck in CA all day and should've been home by now. You owe me.
@USAirways yes it landed at 11:40am. Should not take another 12+ hours to go 40 min drive. Was assured I would have this am.
@USAirways the disappointment was not the bag but that it arrived before me and I was kept 12 hrs without options in IAH
@USAirways can't get back lost time
@USAirways flt 1820 2rsw could

@USAirways  last 2 times I checked a bags they were severally damaged.   No one answers the baggage call line for status?  #chairmanlove
@USAirways this is crazy. Haven't spoken to a human yet. There has to be a better way. http://t.co/mEOAlCIPdD
@usairways can I get assistance on flight #611? Plane just landed from HPN and I have a connection to ATL.
@USAirways you got me home 30 hours after you were supposed to... I was living in an airport, you offered no monetary assistance!
@USAirways I've been trying for DAYS!  And I don't get hung up on until after I get through all the prompts.  What am I to do??
@USAirways Why not have an option for a call back? I'm just sitting here burning through my minutes waiting for an error to be fixed.
@USAirways 2 days and 2 hours on hold the other day. Charge me double for flights and can't even fix it. Worst service I've ever had
@USAirways I've been trying to call you for days and every time the system is overloaded and disconnects. Really?  A glob

@AmericanAir that 2 hr wait became a 4 hr one. And y'all claimed to have sent an email. Yeah, that never happened. #FlyDelta
@AmericanAir it's been 2 hrs of wait on the phone a) worst customer services b) trying to know where my suitcase Is and way MORE 😤#ANGRY
@AmericanAir why don't you take care of @Cowboycerrone flight screw ups
@AmericanAir wht n 👎 experience w/aa reservations #service Tried to be patient with rep and she just disconnects me aa do u record conv.?
@AmericanAir Any info for Cancelled Flightled flight 1046 tomorrow morning out of DFW? Can't get calls through to speak with anyone.
@AmericanAir ticks me off.
@AmericanAir  my fly is 1579  if do you have some news
“@AmericanAir Thanks for info on super large passengers- the extra seat Mr. Big needed was the one i was sitting in already #customerservice
@AmericanAir today, after 6 yrs., no longer my preferred airline.
@AmericanAir Thanks. Two hours to go inflight, then hotel.  Hang in there with all the DFW travel delays A

## Transform into Lowercase

In [80]:
df['text'] = df['text'].str.lower()
df['text']

0                      @virginamerica what @dhepburn said.
1        @virginamerica plus you've added commercials t...
2        @virginamerica i didn't today... must mean i n...
3        @virginamerica it's really aggressive to blast...
4        @virginamerica and it's a really big bad thing...
                               ...                        
14635    @americanair thank you we got on a different f...
14636    @americanair leaving over 20 minutes late flig...
14637    @americanair please bring american airlines to...
14638    @americanair you have my money, you change my ...
14639    @americanair we have 8 ppl so we need 2 know h...
Name: text, Length: 14640, dtype: object

## Convert emoji to text using demoji

In [81]:
import demoji
 
demoji.download_codes()

C:\Users\Asty\AppData\Local\Temp\ipykernel_184468\36668822.py:3: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [82]:
df['text'][0]

'@virginamerica what @dhepburn said.'

In [83]:
def replace_emojis(text):
    emojis = demoji.findall(text)
    for emoji, description in emojis.items():
        text = text.replace(emoji, f' {description} ')
    
    return text

In [84]:
replace_emojis(df['text'][18])

'i  red heart  flying @virginamerica.  smiling face  thumbs up '

In [85]:
df['text'] = df['text'].apply(replace_emojis)

## Removing links

In [86]:
def remove_links(text):
    return re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

df['text'] = df['text'].apply(remove_links)

## Treating stopwords

In [87]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [88]:
df

text  airline_sentiment
0                    @virginamerica what @dhepburn said.                  1
1      @virginamerica plus you've added commercials t...                  2
2      @virginamerica i didn't today... must mean i n...                  1
3      @virginamerica it's really aggressive to blast...                  0
4      @virginamerica and it's a really big bad thing...                  0
...                                                  ...                ...
14635  @americanair thank you we got on a different f...                  2
14636  @americanair leaving over 20 minutes late flig...                  0
14637  @americanair please bring american airlines to...                  1
14638  @americanair you have my money, you change my ...                  0
14639  @americanair we have 8 ppl so we need 2 know h...                  1

[14640 rows x 2 columns]

In [89]:
df['text'] = [nltk.tokenize.word_tokenize(docs) for docs in df['text']]
df

text  airline_sentiment
0         [@, virginamerica, what, @, dhepburn, said, .]                  1
1      [@, virginamerica, plus, you, 've, added, comm...                  2
2      [@, virginamerica, i, did, n't, today, ..., mu...                  1
3      [@, virginamerica, it, 's, really, aggressive,...                  0
4      [@, virginamerica, and, it, 's, a, really, big...                  0
...                                                  ...                ...
14635  [@, americanair, thank, you, we, got, on, a, d...                  2
14636  [@, americanair, leaving, over, 20, minutes, l...                  0
14637  [@, americanair, please, bring, american, airl...                  1
14638  [@, americanair, you, have, my, money, ,, you,...                  0
14639  [@, americanair, we, have, 8, ppl, so, we, nee...                  1

[14640 rows x 2 columns]

In [90]:
df['text'][1]

['@',
 'virginamerica',
 'plus',
 'you',
 "'ve",
 'added',
 'commercials',
 'to',
 'the',
 'experience',
 '...',
 'tacky',
 '.']

In [71]:
nltk.download('stopwords')
all_stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Exclude some words from stopwords. This is a sentiment analysis case. We may need some of these exceptions.

In [93]:
exclude_from_stop = {"not", "no", "never", "none", "isn't", "aren't", "wasn't", "weren't", "doesn't", "don't", 
                     "didn't", "won't", "wouldn't", "can't", "couldn't", "shouldn't", "haven't", "hasn't", 
                     "hadn't", "mightn't", "mustn't", "but", "if", "because", "as", "though", "although",
                     "until", "while", "against", "under", "over", "into", "through", "before", "after", 
                     "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under",
                     "more", "most", "less", "least", "few", "fewer", "many", "several", "enough", "never",
                     "needn't","shan't",'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn',
                     'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn','don'}
stopwrds = [word for word in all_stopwords if word not in exclude_from_stop]

additional_stopwords = ["ain't","'re", "'s","'ve","n't","'m","'d","'tis","y'all"]
stopwrds.extend(additional_stopwords)

print(stopwrds)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'or', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'during', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ma', "ain't", "'re", "'s", "'ve", "n't", "'m", "'d", "'tis", "y'all"]


Finally, remove the stopwords

In [94]:
def remove_stopwords(text):
    return ' '.join([word for word in text if word not in stopwrds])

df['text'] = df['text'].apply(remove_stopwords)
df

text  airline_sentiment
0                      @ virginamerica @ dhepburn said .                  1
1      @ virginamerica plus added commercials to expe...                  2
2      @ virginamerica today ... must mean need to ta...                  1
3      @ virginamerica really aggressive to blast obn...                  0
4                   @ virginamerica really big bad thing                  0
...                                                  ...                ...
14635  @ americanair thank got on different flight to...                  2
14636  @ americanair leaving over 20 minutes late fli...                  0
14637  @ americanair please bring american airlines t...                  1
14638  @ americanair money , change flight , answer p...                  0
14639  @ americanair 8 ppl need 2 know many seats on ...                  1

[14640 rows x 2 columns]

## Removing Punctuations

In [95]:
def remove_punctuation(text):
    whitelist = 'a-zA-Z '
    pattern = f"[^{whitelist}]"
    return re.sub(pattern, '', text)

df['text'] = df['text'].apply(remove_punctuation)
df

text  airline_sentiment
0                          virginamerica  dhepburn said                   1
1       virginamerica plus added commercials to exper...                  2
2       virginamerica today  must mean need to take a...                  1
3       virginamerica really aggressive to blast obno...                  0
4                     virginamerica really big bad thing                  0
...                                                  ...                ...
14635   americanair thank got on different flight to ...                  2
14636   americanair leaving over  minutes late flight...                  0
14637   americanair please bring american airlines to...                  1
14638   americanair money  change flight  answer phon...                  0
14639   americanair  ppl need  know many seats on nex...                  1

[14640 rows x 2 columns]

In [99]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    words = sentence.split()
    pos_tags = nltk.pos_tag(words)
    lemmatized_words = []
    for word, tag in pos_tags:
        wordnet_tag = get_wordnet_pos(tag)
        if wordnet_tag is not None:
            lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_tag)
        else:
            lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)
    return ' '.join(lemmatized_words)

df['text'] = df['text'].apply(lemmatize_sentence)

In [100]:
for text in df['text']:
    print(text)

virginamerica dhepburn say
virginamerica plus added commercial to experience tacky
virginamerica today must mean need to take another trip
virginamerica really aggressive to blast obnoxious entertainment in guest face amp little recourse
virginamerica really big bad thing
virginamerica seriously would pay flight seat play really bad thing fly va
virginamerica yes nearly every time fly vx ear worm win go away
virginamerica really miss prime opportunity men without hat parody
virginamerica well didntbut d
virginamerica amaze arrive hour early good to
virginamerica know suicide second lead cause death among teen
virginamerica lt pretty graphic much good minimal iconography
virginamerica great deal already think nd trip to australia amp even go on st trip yet p
virginamerica virginmedia fly fabulous seductive sky u take stress away from travel
virginamerica thanks
virginamerica sfopdx schedule still mia
virginamerica excite first cross country flight lax to mco heard nothing but great thin

united husband suppose to fly rdu to iad iad to fra leave in hour cancel flighted flight hr wait on phone
united probably least satisfactory airline ever on never fail to disappoint
united flight amaze best flight ever on not kid service incredible
united not sure u call customer care customer care
unite in charge making decision over cancelled flightling flight ua unacceptable in belize right
united fly americanair normally happen to ll let flyer provide feedback thank
united upgrade to first class nice way to fix early mistake thank
united plowing snow gate before plane arrives sit minute after hr flight fail
united saturday morning man die from ebola
united rhonda c atlanta airport redeem guy get u straighten out
united ok do experience family terrible experience kind like robbedexplain
united great job lose one six bag on direct flight know manage to
united bad customer service in denver
united ruin day amp start trip sfo to gig leg change lose pay upgrade amp st class seat amp mov

southwestair thanks send dm to let know if oyu need additional information
southwestair hey friend phone alist number ivr hang up on multiple time not sure to proceed
southwestair strives to customer centric in everything communication advertise customer journey etc anamarketers
southwestair la always traffic but insane ca one issue on flight
southwestair insane traffic in la try to call to schedule flight but on phone minute nothing
southwestair airline trouble amp not get satisfactory customer service contact negotiate resolution
southwestair hey southwest help find flight from nashville washington dc raleigh under on march please
southwestair no longer change trip companion online year get message ca do online
southwestair add rr number to marriott stay hard wo take rr number checkinout marriott phone c not helpful
southwestair spiritairlines fly to toronto kill small
southwestair thank
southwestair traditionally use sledge hammer low take share from drive scalpel approach through d

southwestair spend minute count on hold w today after first minute give up try amp on rd min
southwestair ive on airline everyone else great but soured whole experience hope not next time
southwestair thanks get put on flight tomorrow much faith ll get off ground tomorrow though
southwestair think shot ca find flight arrives before business san on thu really want to go
southwestair woman work counter in philly need to fire due to bad weather along passenger
southwestair earlier reflight book problem flight on get screencap say unspecified error to call swa
southwestair buy early bird pointless move to diff flight bc first one delayed lose boarding position
southwestair problem website try make change w app site but get unspecified error
southwestair paytontaylor love southwest payton taylor
southwestair rdu flight expect to ok afternoon tonight
southwestair super disappoint today first time to stress book flight on another airline while airport
southwestair put on next flight available

usairways anything do to make flight attendant aware aa member benefit embarrass argue happen
usairways question need to talk to someone email give email address thanks
usairways gregm well certainly nothing happen in front scene noaccountability disappointing
usairways hour on tarmac in charleston still ca get response after week unacceptable
usairways customer service best rachel s take great care u phx airport
usairways amp americanair plane in grandcayman
usairways never fly u airway
usairways strike late flight crew trip maintenance worth extra trip less hassle few delay
usairways follow dm
usairways get bump to pm to birmingham delay because pilot schedule enough hour together fail
usairways yes thanks but take forever
usairways as far a delayed go look like tailwind go to make up good news
usairways horrible travel day w airline flight to birmingham cancel flightled w no notification could get on
usairways ask if sop say now but yet flight nothing policy not
usairways while ask 

usairways point baggage claim ticket if scan tracking
usairways no access to dividend mile week automate phone keep tell to call back late flightr frustrate livepersonplease
usairways ca even track bag to see like disappeared
usairways ca get ahold anyone to speak to miss mile over month since travel amp submit twice online
usairways sit on plane minute say another minute something paper work suck
usairways gon na good day to call call on separate occasion past couple week never get to person
usairways spend more on room amp transportation whole flight
usairways ask reimbursement maybe mile add to account inconvenience money to spend
usairways ask reimbursement something like mile add to account told to call customer service
usairways to come home day late flight to find out guy lose luggage
usairways doubt gate x cluster every agent encounter saw include one outright rude put off
usairways to spend more cost flight to get free room transportation to from airport
usairways voucher give

americanair but call hung up
americanair thank
americanair u cud answer plenty inflight steal go on right amp fa wo uncomfortable convo w pas
americanair fine would call left message tell would hour call heard anything yet
americanair flight cancel flightled yesterday early afternoon call be be to wake up tell
americanair in miami agent rachel wong marisol pimentel pleasant in world chaos big thank u
americanair put call back in be still nothing be think might get back to
americanair anyone online help flight leave in hr
americanair cancel flight on amp flight hr longer amp layover late flight to find reasonably price alternative
americanair know may also work from home thanks nothing
americanair call ep line hour late flightr call back sat on hold over hour to u airway cut off
americanair flight cancel flightled need to get back to cleveland option
americanair way could put up in hotel night if not to sleep airport tonight
americanair mallowfairy many time exactly call leave number ca

In [103]:
RANDOM_SEED = 42
df_train, df_test = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)

In [107]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(df_train['text'])
X_test_tfidf = tfidf.transform(df_test['text'])

In [108]:
X_train_tfidf

<10248x9165 sparse matrix of type '<class 'numpy.float64'>'
	with 112549 stored elements in Compressed Sparse Row format>

In [109]:
X_test_tfidf

<4392x9165 sparse matrix of type '<class 'numpy.float64'>'
	with 46628 stored elements in Compressed Sparse Row format>

In [ ]:
clf = xgb.XGBClassifier()
clf.fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
accuracy_base = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy_base)
print(confusion_matrix(y_test, y_pred))